In [1]:
from helpers.utilities import *
%run helpers/notebook_setup.ipynb

# RNA-Seq

In [4]:
!ls data/clean/rna/

all_samples.csv  definite_tbm_against_all.csv  definite_tbm_subset.csv


In [ ]:
gene_level_path = 'data/clean/rna/all_samples.csv'
# TODO: extract in a separate notebook? 
isoforms_path = 'data/raw/CSF_Isoform_merged_counts.txt'

In [5]:
rna_levels = read_csv(gene_level_path, index_col=[0,1])

In [6]:
assert not rna_levels.isnull().any().any()

In [7]:
rna_levels

,,001.TMD,006.CM,012.BM,016.CM,017.TMD,...,174.CM,011.TMR,043.TMS,078.CM,261.CM
ensembl_id,ensembl_gene_id,,,,,,,,,,,
ENSG00000000003,TSPAN6,11.596119,7.806308,1.391555,39.347366,24.765008,...,4.431071,51.810470,35.628629,0.000000,20.705553
ENSG00000000005,TNMD,0.000000,0.000000,0.000000,2.459210,0.000000,...,0.000000,11.102244,0.000000,0.000000,4.141111
ENSG00000000419,DPM1,0.000000,31.225230,36.876198,0.000000,72.313823,...,25.478656,11.102244,0.000000,161.520987,39.754662
ENSG00000000457,SCYL3,127.557309,145.717740,86.276387,95.909204,123.825040,...,121.854440,96.219444,3.958737,92.395480,125.889763
ENSG00000000460,C1orf112,46.384476,0.000000,50.791744,51.643418,14.859005,...,33.233029,49.960096,38.267787,0.000000,30.644219
...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000284543,LINC01226,92.768952,200.361893,27.135315,177.063146,30.708610,...,175.027287,170.234402,318.018506,131.406905,52.177994
ENSG00000284546,SSU72P3,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,14.802991,0.000000,0.000000,0.000000
ENSG00000284552,AC106774.10,0.000000,0.000000,0.000000,17.214473,0.000000,...,11.077676,9.251870,0.000000,27.376439,0.828222


In [8]:
rna_levels = rna_levels.reset_index(level='ensembl_gene_id', drop=True)

In [9]:
%%R -i rna_levels
#pheatmap(rna_levels, show_colnames=FALSE, show_rownames=FALSE)

NULL


## Isoforms data

In [42]:
isoforms = read_table(isoforms_path)
definite_tbm_subset = read_excel('data/raw/RNA-Seq/DefiniteTBM_CM_VM/DefTBM_CM_VM.xlsx')
all_gene_level = read_table('data/raw/RNA-Seq/AllSamples/NormalisedCounts_AllSamples_ConditionFiltered.txt')

In [50]:
# remainder: AllSamples contain the TBM subset samples 
set(definite_tbm_subset.columns) - set(all_gene_level.columns)

{'Description',
 'Feature.1',
 'baseMean',
 'lfcSE',
 'log2FoldChange',
 'padj',
 'pvalue',
 'stat'}

In [63]:
new_samples = set(isoforms.columns.str.replace('-', '.')) - set(all_gene_level.columns)
new_samples = Series(list(new_samples)).str.replace('.', '-')

In [64]:
new_samples

0    Unnamed: 0
1       167-TMR
2       158-TMD
3       175-TMD
4       093-TMD
5       074-TMS
6       128-TMD
7        233-CM
dtype: object

In [65]:
def ratio_of_zeros(data):
    return (data == 0).sum() / len(data)

In [66]:
ratio_of_zeros(isoforms['006-CM'])

0.7152995974582651

In [67]:
ratio_of_zeros(isoforms['167-TMR'])

0.9889275926799642

In [69]:
ratio_of_zeros(isoforms[new_samples])

Unnamed: 0    0.000000
167-TMR       0.988928
158-TMD       0.994855
175-TMD       0.981073
093-TMD       0.962913
074-TMS       0.973112
128-TMD       0.980315
233-CM        0.974618
dtype: float64

In [70]:
ratio_of_zeros(isoforms[isoforms.columns.difference(new_samples)]).describe()

count    49.000000
mean      0.682162
std       0.096854
min       0.481524
25%       0.613379
50%       0.666839
75%       0.756021
max       0.881521
dtype: float64